# Necessary Upgrades
Run the cells in this section to make sure you have the latest version of sklearn and joblib.

Restart your kernel after installing.

In [1]:
## Update sklearn to prevent version mismatches
# !pip install sklearn --upgrade

In [2]:
## install joblib. This will be used to save your model. 
# !pip install joblib
## Restart your kernel after installing 

# Import Dependencies

In [3]:
import warnings
warnings.simplefilter('ignore')

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

import joblib

# Read the CSV and Perform Basic Data Cleaning

In [4]:
df = pd.read_csv("data/exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
# df.head()

In [5]:
# drop the error columns
df = df.loc[:, ~df.columns.str.contains('_err')]
df.columns

# column definitions is available at the end of this notebook.

Index(['koi_disposition', 'koi_fpflag_nt', 'koi_fpflag_ss', 'koi_fpflag_co',
       'koi_fpflag_ec', 'koi_period', 'koi_time0bk', 'koi_impact',
       'koi_duration', 'koi_depth', 'koi_prad', 'koi_teq', 'koi_insol',
       'koi_model_snr', 'koi_tce_plnt_num', 'koi_steff', 'koi_slogg',
       'koi_srad', 'ra', 'dec', 'koi_kepmag'],
      dtype='object')

# Select your features (columns) and y-variable.
Column definitions are available at the end of this notebook.

Use `koi_disposition` for the y values

In [6]:
y = df[['koi_disposition']]

In [7]:
# Set features. This will also be used as your x values.
selected_features = df[['koi_fpflag_nt', 'koi_prad', 'koi_teq', 'koi_srad']]
feature_names = selected_features.columns
# selected_features.head()

# Create a Train Test Split

In [8]:
X_train, X_test, y_train, y_test = train_test_split(selected_features, y, random_state=42)
# X_train.head()

# Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [9]:
# Scale your data
X_scaler = StandardScaler().fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Train the Model



In [10]:
rfc = RandomForestClassifier()
rfc.fit(X_train_scaled, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [11]:
print(f"Training Data Score: {rfc.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {rfc.score(X_test_scaled, y_test)}")

Training Data Score: 1.0
Testing Data Score: 0.7008009153318078


In [12]:
importances = rfc.feature_importances_
sorted(zip(importances, feature_names), reverse=True)

[(0.3698898229867709, 'koi_prad'),
 (0.2829304099161924, 'koi_teq'),
 (0.2241332343529633, 'koi_srad'),
 (0.12304653274407341, 'koi_fpflag_nt')]

In [13]:
# Calculate classification report
predictions = rfc.predict(X_test_scaled)
print(classification_report(y_test, predictions))

                precision    recall  f1-score   support

     CANDIDATE       0.52      0.51      0.51       411
     CONFIRMED       0.62      0.61      0.61       484
FALSE POSITIVE       0.83      0.85      0.84       853

      accuracy                           0.70      1748
     macro avg       0.66      0.65      0.65      1748
  weighted avg       0.70      0.70      0.70      1748



# Hyperparameter Tuning

Use `GridSearchCV` to tune the model's parameters

In [14]:
# Create the GridSearchCV model

param_grid = { 
    'n_estimators': [200],
    "min_samples_split": [2, 5, 10, 15, 100],
    'max_depth' : [5, 8, 15, 25, 30],
    "min_samples_leaf": [1, 2, 5, 10] 
}

rfc_grid = GridSearchCV(estimator=rfc, param_grid=param_grid, cv= 5, verbose = 2)

In [15]:
# Train the model with GridSearch
best_rfc_model = rfc_grid.fit(X_train_scaled, y_train)

Fitting 5 folds for each of 100 candidates, totalling 500 fits
[CV] max_depth=5, min_samples_leaf=1, min_samples_split=2, n_estimators=200 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  max_depth=5, min_samples_leaf=1, min_samples_split=2, n_estimators=200, total=   0.6s
[CV] max_depth=5, min_samples_leaf=1, min_samples_split=2, n_estimators=200 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.5s remaining:    0.0s


[CV]  max_depth=5, min_samples_leaf=1, min_samples_split=2, n_estimators=200, total=   0.6s
[CV] max_depth=5, min_samples_leaf=1, min_samples_split=2, n_estimators=200 
[CV]  max_depth=5, min_samples_leaf=1, min_samples_split=2, n_estimators=200, total=   0.6s
[CV] max_depth=5, min_samples_leaf=1, min_samples_split=2, n_estimators=200 
[CV]  max_depth=5, min_samples_leaf=1, min_samples_split=2, n_estimators=200, total=   0.5s
[CV] max_depth=5, min_samples_leaf=1, min_samples_split=2, n_estimators=200 
[CV]  max_depth=5, min_samples_leaf=1, min_samples_split=2, n_estimators=200, total=   0.6s
[CV] max_depth=5, min_samples_leaf=1, min_samples_split=5, n_estimators=200 
[CV]  max_depth=5, min_samples_leaf=1, min_samples_split=5, n_estimators=200, total=   0.5s
[CV] max_depth=5, min_samples_leaf=1, min_samples_split=5, n_estimators=200 
[CV]  max_depth=5, min_samples_leaf=1, min_samples_split=5, n_estimators=200, total=   0.8s
[CV] max_depth=5, min_samples_leaf=1, min_samples_split=5, n_es

...

[CV]  max_depth=25, min_samples_leaf=2, min_samples_split=10, n_estimators=200, total=   0.9s
[CV] max_depth=25, min_samples_leaf=2, min_samples_split=15, n_estimators=200 
[CV]  max_depth=25, min_samples_leaf=2, min_samples_split=15, n_estimators=200, total=   1.1s
[CV] max_depth=25, min_samples_leaf=2, min_samples_split=15, n_estimators=200 
[CV]  max_depth=25, min_samples_leaf=2, min_samples_split=15, n_estimators=200, total=   0.9s
[CV] max_depth=25, min_samples_leaf=2, min_samples_split=15, n_estimators=200 
[CV]  max_depth=25, min_samples_leaf=2, min_samples_split=15, n_estimators=200, total=   0.9s
[CV] max_depth=25, min_samples_leaf=2, min_samples_split=15, n_estimators=200 
[CV]  max_depth=25, min_samples_leaf=2, min_samples_split=15, n_estimators=200, total=   0.9s
[CV] max_depth=25, min_samples_leaf=2, min_samples_split=15, n_estimators=200 
[CV]  max_depth=25, min_samples_leaf=2, min_samples_split=15, n_estimators=200, total=   0.9s
[CV] max_depth=25, min_samples_leaf=2, mi

[CV]  max_depth=25, min_samples_leaf=10, min_samples_split=10, n_estimators=200, total=   0.8s
[CV] max_depth=25, min_samples_leaf=10, min_samples_split=10, n_estimators=200 
[CV]  max_depth=25, min_samples_leaf=10, min_samples_split=10, n_estimators=200, total=   0.8s
[CV] max_depth=25, min_samples_leaf=10, min_samples_split=10, n_estimators=200 
[CV]  max_depth=25, min_samples_leaf=10, min_samples_split=10, n_estimators=200, total=   0.8s
[CV] max_depth=25, min_samples_leaf=10, min_samples_split=15, n_estimators=200 
[CV]  max_depth=25, min_samples_leaf=10, min_samples_split=15, n_estimators=200, total=   0.8s
[CV] max_depth=25, min_samples_leaf=10, min_samples_split=15, n_estimators=200 
[CV]  max_depth=25, min_samples_leaf=10, min_samples_split=15, n_estimators=200, total=   0.8s
[CV] max_depth=25, min_samples_leaf=10, min_samples_split=15, n_estimators=200 
[CV]  max_depth=25, min_samples_leaf=10, min_samples_split=15, n_estimators=200, total=   0.8s
[CV] max_depth=25, min_samples

[CV]  max_depth=30, min_samples_leaf=2, min_samples_split=10, n_estimators=200, total=   0.9s
[CV] max_depth=30, min_samples_leaf=2, min_samples_split=10, n_estimators=200 
[CV]  max_depth=30, min_samples_leaf=2, min_samples_split=10, n_estimators=200, total=   0.9s
[CV] max_depth=30, min_samples_leaf=2, min_samples_split=10, n_estimators=200 
[CV]  max_depth=30, min_samples_leaf=2, min_samples_split=10, n_estimators=200, total=   1.1s
[CV] max_depth=30, min_samples_leaf=2, min_samples_split=10, n_estimators=200 
[CV]  max_depth=30, min_samples_leaf=2, min_samples_split=10, n_estimators=200, total=   0.9s
[CV] max_depth=30, min_samples_leaf=2, min_samples_split=10, n_estimators=200 
[CV]  max_depth=30, min_samples_leaf=2, min_samples_split=10, n_estimators=200, total=   1.1s
[CV] max_depth=30, min_samples_leaf=2, min_samples_split=15, n_estimators=200 
[CV]  max_depth=30, min_samples_leaf=2, min_samples_split=15, n_estimators=200, total=   1.2s
[CV] max_depth=30, min_samples_leaf=2, mi

[CV]  max_depth=30, min_samples_leaf=10, min_samples_split=5, n_estimators=200, total=   0.9s
[CV] max_depth=30, min_samples_leaf=10, min_samples_split=5, n_estimators=200 
[CV]  max_depth=30, min_samples_leaf=10, min_samples_split=5, n_estimators=200, total=   1.1s
[CV] max_depth=30, min_samples_leaf=10, min_samples_split=10, n_estimators=200 
[CV]  max_depth=30, min_samples_leaf=10, min_samples_split=10, n_estimators=200, total=   0.8s
[CV] max_depth=30, min_samples_leaf=10, min_samples_split=10, n_estimators=200 
[CV]  max_depth=30, min_samples_leaf=10, min_samples_split=10, n_estimators=200, total=   0.8s
[CV] max_depth=30, min_samples_leaf=10, min_samples_split=10, n_estimators=200 
[CV]  max_depth=30, min_samples_leaf=10, min_samples_split=10, n_estimators=200, total=   0.8s
[CV] max_depth=30, min_samples_leaf=10, min_samples_split=10, n_estimators=200 
[CV]  max_depth=30, min_samples_leaf=10, min_samples_split=10, n_estimators=200, total=   0.8s
[CV] max_depth=30, min_samples_le

[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:  6.3min finished


In [16]:
print(rfc_grid.best_params_)
print(rfc_grid.best_score_)
print(rfc_grid.best_estimator_)

{'max_depth': 15, 'min_samples_leaf': 5, 'min_samples_split': 2, 'n_estimators': 200}
0.7354543403750572
RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=15, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=5, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)


# Test Model

In [17]:
predictions = rfc_grid.predict(X_test_scaled)
predicted = predictions[:20]
actual = y_test["koi_disposition"][:20].tolist()
pd.DataFrame({"Predictions": predicted, "Actual": actual}).reset_index(drop=True)

,Predictions,Actual
0,FALSE POSITIVE,FALSE POSITIVE
1,CANDIDATE,CANDIDATE
2,FALSE POSITIVE,FALSE POSITIVE
3,FALSE POSITIVE,FALSE POSITIVE
4,FALSE POSITIVE,FALSE POSITIVE
5,CONFIRMED,CONFIRMED
6,CANDIDATE,CANDIDATE
7,CANDIDATE,CANDIDATE
8,CANDIDATE,CANDIDATE
9,FALSE POSITIVE,FALSE POSITIVE


In [18]:
print(classification_report(y_test, predictions))

                precision    recall  f1-score   support

     CANDIDATE       0.57      0.51      0.54       411
     CONFIRMED       0.65      0.67      0.66       484
FALSE POSITIVE       0.84      0.86      0.85       853

      accuracy                           0.73      1748
     macro avg       0.69      0.68      0.68      1748
  weighted avg       0.72      0.73      0.73      1748



In [19]:
print(f"Training Data Score (Best): {best_rfc_model.score(X_train_scaled, y_train)}")
print(f"Testing Data Score (Best): {best_rfc_model.score(X_test_scaled, y_test)}")

Training Data Score (Best): 0.8212855235552164
Testing Data Score (Best): 0.7282608695652174


# Save the Model

In [20]:
filename = 'RFC_model.h5'
joblib.dump(best_rfc_model, filename)

['RFC_model.h5']

In [21]:
loaded_model = joblib.load("RFC_model.h5")
loaded_model.score(X_test_scaled, y_test)

0.7282608695652174

# Column Details

### Exoplanet Archive Information
* 'koi_disposition': The pipeline flag that designates the most probable physical explanation of the KOI

### Project Disposition Columns
* 'koi_fpflag_nt': A KOI whose light curve is not consistent with that of a transiting planet.
* 'koi_fpflag_ss': A KOI that is observed to have a significant secondary event, transit shape, or out-of-eclipse variability, which indicates that the transit-like event is most likely caused by an eclipsing binary.
* 'koi_fpflag_co': The source of the signal is from a nearby star, as inferred by measuring the centroid location of the image both in and out of transit, or by the strength of the transit signal in the target's outer (halo) pixels as compared to the transit signal from the pixels in the optimal (or core) aperture.
* 'koi_fpflag_ec': The KOI shares the same period and epoch as another object and is judged to be the result of flux contamination in the aperture or electronic crosstalk.

### Transit Properties
Transit parameters delivered by the Kepler Project are typically best-fit parameters produced by a Mandel-Agol (2002) fit to a multi-quarter Kepler light curve, assuming a linear orbital ephemeris. Some of the parameters listed below are fit directly, other are derived from the best-fit parameters. Limb-darkening coefficients are fixed and pre-calculated from host star properties. Orbital Period, Transit Epoch, Planet-Star Radius Ratio, Planet-Star Distance over Star Radius and Impact Parameter are the free parameters in the fit. Matrix covariances are adopted as errors to the fit parameters; they therefore ignore the effects of correlation between the fit parameters and are likely to be underestimates.

See the links in the Purpose of KOI document for each activity table for more details on the fits for each delivery.

Scaled planetary parameters combine the dimensionless fit parameters with physical stellar parameters to produce planet characteristics in physical units.

Best-fit planetary transit parameters are typically normalized to the size of the host star. Physical planet parameters may be derived by scaling to the star's size and temperature. Transit parameters also depend weakly upon the limb-darkening coefficients which are derived from the stellar parameters (e.g., Claret and Bloemen 2011).

* 'koi_period': The interval between consecutive planetary transits.
* 'koi_time0bk': The time corresponding to the center of the first detected transit in Barycentric Julian Day (BJD) minus a constant offset of 2,454,833.0 days. The offset corresponds to 12:00 on Jan 1, 2009 UTC.
* 'koi_impact': The sky-projected distance between the center of the stellar disc and the center of the planet disc at conjunction, normalized by the stellar radius
* 'koi_duration': The duration of the observed transits. Duration is measured from first contact between the planet and star until last contact. Contact times are typically computed from a best-fit model produced by a Mandel-Agol (2002) model fit to a multi-quarter Kepler light curve, assuming a linear orbital ephemeris.
* 'koi_depth': The fraction of stellar flux lost at the minimum of the planetary transit. Transit depths are typically computed from a best-fit model produced by a Mandel-Agol (2002) model fit to a multi-quarter Kepler light curve, assuming a linear orbital ephemeris.
* 'koi_prad': The radius of the planet. Planetary radius is the product of the planet star radius ratio and the stellar radius.
* 'koi_teq': Approximation for the temperature of the planet. 
* 'koi_insol': Insolation flux is another way to give the equilibrium temperature.

### Threshold-Crossing Event (TCE) Information
The Transiting Planet Search (TPS) module of the Kepler data analysis pipeline performs a detection test for planet transits in the multi-quarter, gap-filled flux time series. The TPS module detrends each quarterly PDC light curve to remove edge effects around data gaps and then combines the data segments together, filling gaps with interpolated data so as to condition the flux time series for a matched filter. The module applies an adaptive, wavelet-based matched filter (Jenkins 2002, Jenkins et al. 2010 and Tenenbaum et al. (2012)) to perform a joint characterization of observation noise and detection of transit-like features in the light curve.

The TPS module estimates the Power Spectral Density of the flux time series as a function in time. This provides coefficients for a whitening filter to accommodate non-stationary, non-white noise and yields Single Event Statistic (SES) time series components. These can be interpreted as measurements of the statistical significance of the presence of a transit of trial duration at each point in the time series.

Single Event Statistics are folded at each trial orbital period and the maximum Multiple Event Statistic (MES) is obtained over all trial periods and phases. The MES estimates the signal to noise ratio of the putative transit-like sequence against the measurement noise. The MES threshold for defining the sample of Threshold Crossing Events (TCEs) is provided within the Release Notes. For reference, a lower MES threshold of 7.1σ limits the number of false positives in the TCE sample due to statistical random noise to less than 1 over the primary mission (Jenkins, Caldwell and Borucki 2002).

* 'koi_model_snr': Transit depth normalized by the mean uncertainty in the flux during the transits.
* 'koi_tce_plnt_num': TCE Planet Number federated to the KOI.

### Stellar Parameters
Stellar effective temperature, surface gravity, metallicity, radius, mass, and age should comprise a consistent set. Associated error estimates are 1-σ uncertainties.

* 'koi_steff': The photospheric temperature of the star.
* 'koi_slogg': The base-10 logarithm of the acceleration due to gravity at the surface of the star.
* 'koi_srad': The photospheric radius of the star

### KIC Parameters
* 'ra': KIC Right Ascension
* 'dec': KIC Declination
* 'koi_kepmag': Kepler-band (mag)